<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE4-Sensor-Pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""
Simple Sensor Pre-Training System
=================================

Trains LSTM Autoencoders for each sensor and saves models with baseline statistics.

Usage:
    python sensor_pretraining.py
"""

import numpy as np
import os
import pickle
from datetime import datetime
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


def build_lstm_autoencoder(window_length: int, latent_dim: int = 4) -> Model:
    """Build LSTM Autoencoder."""
    inputs = Input(shape=(window_length, 1))

    # Encoder
    encoded = LSTM(latent_dim,  return_sequences=False)(inputs)

    # Decoder
    decoded = RepeatVector(window_length)(encoded)
    decoded = LSTM(latent_dim,return_sequences=True)(decoded)
    outputs = TimeDistributed(Dense(1, activation='linear'))(decoded)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001, clipnorm=1.0), loss='mse')

    return model


def train_sensor_model(sensor_data, sensor_id, base_path, window_length):
    """Train model for one sensor."""

    print(f"\nTraining sensor {sensor_id}...")
    print(f"Data shape: {sensor_data.shape}")

    # Split data
    n_samples = len(sensor_data)
    n_train = int(0.8 * n_samples)

    X_train = sensor_data[:n_train]
    X_val = sensor_data[n_train:]

    print(f"Train: {len(X_train)}, Val: {len(X_val)}")

    # Create directories
    sensor_dir = os.path.join(base_path, 'sensor', 'model')
    checkpoint_dir = os.path.join(sensor_dir, 'checkpoints')
    os.makedirs(sensor_dir, exist_ok=True)
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Build model
    model = build_lstm_autoencoder(window_length)

    # Setup callbacks
    checkpoint_path = os.path.join(checkpoint_dir, f'sensor_{sensor_id}_best.h5')

    callbacks = [
        ModelCheckpoint(
            filepath=checkpoint_path,
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=False,
            mode='min',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=8,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-6,
            verbose=1
        )
    ]

    # Train model
    print("Starting training with checkpointing...")
    history = model.fit(
        X_train, X_train,
        validation_data=(X_val, X_val),
        epochs=50,
        batch_size=64,
        callbacks=callbacks,
        verbose=1
    )

    print(f"Training completed. Best model saved to: {checkpoint_path}")

    # Compute baseline reconstruction errors on validation set
    print("Computing baseline errors...")
    val_predictions = model.predict(X_val, verbose=0)
    baseline_errors = []

    for i in range(len(X_val)):
        error = mean_squared_error(X_val[i].flatten(), val_predictions[i].flatten())
        baseline_errors.append(error)

    baseline_stats = {
        'mean': float(np.mean(baseline_errors)),
        'std': float(np.std(baseline_errors)) + 1e-8,
        'q95': float(np.percentile(baseline_errors, 95)),
        'q99': float(np.percentile(baseline_errors, 99)),
        'median': float(np.median(baseline_errors)),
        'mad': float(np.median(np.abs(baseline_errors - np.median(baseline_errors)))),
        'baseline_errors': baseline_errors
    }

    # Save final model and metadata
    model_path = os.path.join(sensor_dir, f'sensor_{sensor_id}_model.h5')
    metadata_path = os.path.join(sensor_dir, f'sensor_{sensor_id}_metadata.pkl')

    model.save(model_path)

    metadata = {
        'sensor_id': sensor_id,
        'window_length': window_length,
        'baseline_stats': baseline_stats,
        'trained_at': datetime.now(),
        'epochs_trained': len(history.history['loss']),
        'final_val_loss': float(history.history['val_loss'][-1]),
        'checkpoint_path': checkpoint_path
    }

    with open(metadata_path, 'wb') as f:
        pickle.dump(metadata, f)

    print(f"✅ Sensor {sensor_id} completed")
    print(f"   Final model: {model_path}")
    print(f"   Best checkpoint: {checkpoint_path}")
    print(f"   Baseline error: {baseline_stats['mean']:.6f} ± {baseline_stats['std']:.6f}")

    return baseline_stats


def main():
    """Main training function."""

    data_path = r'/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-10Sec-DIRECT-VAR.npy'
    labelpath = r'/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/window_labels_3class.npy'
    train_mask_path = r'/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/train_mask.npy'

    base_path = r'/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/'

    print("🚀 Simple Sensor Pre-Training")
    print("=" * 40)

    # Load data
    print("Loading data...")
    data = np.load(data_path)
    label = np.load(labelpath)
    train_mask = np.load(train_mask_path)

    # AE mask
    ae_mask = np.logical_and(train_mask, label == 0)
    print("Total AE windows:", np.sum(ae_mask))

    training_data = data[ae_mask]
    print(f"AE data shape: {training_data.shape}")

    batch_size, window_length, num_sensors = training_data.shape
    print(f"Will train {num_sensors} sensors")

    results = {}
    successful = 0

    # Sequential sensor training
    for sensor_id in range(num_sensors):

        model_path = os.path.join(base_path, "sensor", "model", f"sensor_{sensor_id}_model.h5")
        ckpt_path = os.path.join(base_path, "sensor", "model", "checkpoints", f"sensor_{sensor_id}_best.h5")

        if os.path.exists(model_path) and os.path.exists(ckpt_path):
            print(f"⏭️ Sensor {sensor_id} already trained. Skipping.")
            continue

        print(f"\n🔧 Starting training for Sensor {sensor_id}...")
        sensor_data = training_data[:, :, sensor_id:sensor_id+1]

        try:
            baseline_stats = train_sensor_model(sensor_data, sensor_id, base_path, window_length)
            results[sensor_id] = {"success": True, "baseline_stats": baseline_stats}
            successful += 1
        except Exception as e:
            print(f"❌ Failed training sensor {sensor_id}: {e}")
            results[sensor_id] = {"success": False, "error": str(e)}

    # Final summary
    print("\n📊 TRAINING SUMMARY")
    print("=" * 40)
    print(f"Trained this run: {successful}/{num_sensors}")
    print(f"Models saved to: {os.path.join(base_path, 'sensor', 'model')}")

    summary_path = os.path.join(base_path, 'sensor', 'model', 'training_summary.pkl')
    with open(summary_path, "wb") as f:
        pickle.dump({
            "results": results,
            "training_data_shape": training_data.shape,
            "successful_sensors": successful,
            "timestamp": datetime.now()
        }, f)

    print(f"💾 Summary saved: {summary_path}")
    print("✅ Pre-training completed!")


if __name__ == "__main__":
    main()



🚀 Simple Sensor Pre-Training
Loading data...
Total AE windows: 432142
AE data shape: (432142, 100, 12)
Will train 12 sensors

🔧 Starting training for Sensor 0...

Training sensor 0...
Data shape: (432142, 100, 1)
Train: 345713, Val: 86429
Starting training with checkpointing...
Epoch 1/50
5402/5402 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2898
Epoch 1: val_loss improved from inf to 0.26800, saving model to /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_0_best.h5


5402/5402 ━━━━━━━━━━━━━━━━━━━━ 117s 16ms/step - loss: 0.2898 - val_loss: 0.2680 - learning_rate: 0.0010
Epoch 2/50
5401/5402 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2243
Epoch 2: val_loss did not improve from 0.26800
5402/5402 ━━━━━━━━━━━━━━━━━━━━ 66s 12ms/step - loss: 0.2243 - val_loss: 0.2688 - learning_rate: 0.0010
Epoch 3/50
5401/5402 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2174
Epoch 3: val_loss improved from 0.26800 to 0.19521, saving model to /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_0_best.h5


5402/5402 ━━━━━━━━━━━━━━━━━━━━ 67s 12ms/step - loss: 0.2174 - val_loss: 0.1952 - learning_rate: 0.0010
Epoch 4/50
5401/5402 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1681
Epoch 4: val_loss did not improve from 0.19521
5402/5402 ━━━━━━━━━━━━━━━━━━━━ 66s 12ms/step - loss: 0.1681 - val_loss: 0.3020 - learning_rate: 0.0010
Epoch 5/50
5401/5402 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2763
Epoch 5: val_loss did not improve from 0.19521
5402/5402 ━━━━━━━━━━━━━━━━━━━━ 66s 12ms/step - loss: 0.2763 - val_loss: 0.2961 - learning_rate: 0.0010
Epoch 6/50
5401/5402 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2583
Epoch 6: val_loss improved from 0.19521 to 0.17248, saving model to /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_0_best.h5


5402/5402 ━━━━━━━━━━━━━━━━━━━━ 67s 12ms/step - loss: 0.2583 - val_loss: 0.1725 - learning_rate: 0.0010
Epoch 7/50
5401/5402 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2401
Epoch 7: val_loss did not improve from 0.17248
5402/5402 ━━━━━━━━━━━━━━━━━━━━ 66s 12ms/step - loss: 0.2401 - val_loss: 0.2671 - learning_rate: 0.0010
Epoch 8/50
5401/5402 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: nan
Epoch 8: val_loss did not improve from 0.17248
5402/5402 ━━━━━━━━━━━━━━━━━━━━ 66s 12ms/step - loss: nan - val_loss: nan - learning_rate: 0.0010
Epoch 9/50
5401/5402 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: nan
Epoch 9: val_loss did not improve from 0.17248
5402/5402 ━━━━━━━━━━━━━━━━━━━━ 66s 12ms/step - loss: nan - val_loss: nan - learning_rate: 0.0010
Epoch 10/50
5401/5402 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: nan
Epoch 10: val_loss did not improve from 0.17248
5402/5402 ━━━━━━━━━━━━━━━━━━━━ 66s 12ms/step - loss: nan - val_loss: nan - learning_rate: 0.0010
Epoch 11/50
5401/5402 ━━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')